# Rough Notebook to be changed to pct reduction
## Last Updated: 10-30-2020

In [1]:
# Importing the necessary libraries

import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
from sklearn.model_selection import train_test_split
import json
import warnings
from os import listdir
from os.path import isfile,join
import datetime
import requests
from sklearn.metrics import r2_score, accuracy_score
import catboost
import matplotlib as mpl
import sys
sys.path.append('.')
from src.utils import collect_data
from src.utils import eda


In [2]:
import importlib
importlib.reload(collect_data)
collect_datasets = collect_data.DataCollection()
data_analysis = eda.DataAnalysis()

In [3]:
df_mobility = collect_datasets.get_mobility_data(download=False)
df_county_spending = collect_datasets.get_spend_data()
df_covid = collect_datasets.combine_covid_data()
df_weather = collect_datasets.combine_weather(df_mobility,df_covid)
df_county = collect_datasets.combined_county_data()

C:\Work\projects\mobility\US_Mobility_BTS\src\utils\collect_data.py:51: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df_mobility['Wk #'] = (df_mobility.date.dt.week.astype(str) )


In [4]:
def get_df_2019(df_weather, df_mob,df_covid,df_demographics,year=2019):
    df_mob = df_mob.dropna()
    thr_date = datetime.datetime.strptime('2020-01-01', '%Y-%m-%d')
    if year==2020:
        df_mob_prep = pd.merge(left = df_mob, right = df_covid, on = ['fips', 'date'], how='inner')
        df_mob_prep = df_mob_prep[df_mob_prep.date <thr_date]
    else:
        df_mob_prep = df_mob[(df_mob.date < thr_date)]
        
    df_mob_weather = pd.merge(left = df_mob_prep, right = df_weather, on =['date', 'fips'])

    return df_mob_weather

In [5]:
df_2019 = get_df_2019(df_weather, df_mobility, df_covid, df_demographics=df_county)
df_2019_fil = df_2019[df_2019['State Postal Code'] != 'AK']


### Time Series

In [618]:
targ_var = 'mobility_per_person'
ts_base_map = data_analysis.plot_ts_map(df_2019_fil,targ_var,q_filter=True)

C:\Work\projects\mobility\US_Mobility_BTS\src\utils\eda.py:222: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['id'] = df['id'].astype(int)


In [ ]:
targ_var = 'trips_per_person'
trips_per_person_ts_map = data_analysis.plot_ts_map(df_2019_fil,targ_var,q_filter=True)